# Extract stability operators

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.append(os.environ["PETSC_DIR"]+'/lib/petsc/bin/')
from PetscBinaryIO import PetscBinaryIO

In [3]:
import case

import scipy.linalg as la
import scipy.sparse as sp
import scipy.sparse.linalg as spla

np.set_printoptions(linewidth=130)

In [4]:
solver = case.solver()

In [5]:
%%time
bf, solver = case.base_flow(verbose = True)

   k   residual_x   residual_f    cyl[-]_fx    cyl[-]_fy    cyl[0]_fx    cyl[0]_fy    cyl[+]_fx    cyl[+]_fy 
   1  5.79989e-01  1.71841e+01  4.38533e+00  8.63622e-01  4.38533e+00  8.63622e-01  4.38533e+00  8.63622e-01 
   2  3.94615e-01  2.95182e-01  2.34880e+00  1.02337e+00  2.34880e+00  1.02337e+00  2.34880e+00  1.02337e+00 
   3  1.25234e-01  8.32867e-02  2.42449e+00  9.84748e-01  2.42449e+00  9.84748e-01  2.42449e+00  9.84748e-01 
   4  4.47184e-02  4.84048e-02  2.42439e+00  1.00209e+00  2.42439e+00  1.00209e+00  2.42439e+00  1.00209e+00 
   5  1.35552e-02  6.29178e-03  2.42420e+00  1.00215e+00  2.42420e+00  1.00215e+00  2.42420e+00  1.00215e+00 
   6  8.83266e-04  5.58616e-04  2.42420e+00  1.00215e+00  2.42420e+00  1.00215e+00  2.42420e+00  1.00215e+00 
   7  1.79048e-06  1.36630e-06  2.42420e+00  1.00215e+00  2.42420e+00  1.00215e+00  2.42420e+00  1.00215e+00 
   8  1.06144e-11  7.41403e-12  2.42420e+00  1.00215e+00  2.42420e+00  1.00215e+00  2.42420e+00  1.00215e+00 
CPU times:

In [6]:
A, B = (M[0].tocsr() for M in solver.propagator(fractionalStep=False))

N = solver.fluid.linearized_advection(*solver.reshape(*solver.unpack(bf))[:2], 
                                       *case.boundary_conditions(solver)[:2]).tocsr()
N = sp.block_diag((N, sp.coo_matrix((A.shape[0] - N.shape[0],)*2)))

In [7]:
MM = A
CC = -B + 1.5*N
KK = -0.5*N

sU = solver.fluid.u.size//3
sV = solver.fluid.v.size//3
sP = sV - 1
sL = solver.solids[0].l

idx1 = np.r_[                                 : sU, 
             3*sU                             : 3*sU + sV, 
             3*sU + 3*sV                      : 3*sU + 3*sV + sP,
             3*sU + 3*sV + sP + 2*sV          : 3*sU + 3*sV + sP + 2*sV + 2*sL]
idx2 = np.r_[sU                                  : 2*sU, 
             3*sU + sV                           : 3*sU + 2*sV, 
             3*sU + 3*sV + sP                    : 3*sU + 3*sV + sP + sV,
             3*sU + 3*sV + sP + 2*sV + 2*sL        : 3*sU + 3*sV + sP + 2*sV + 4*sL]
idx3 = np.r_[2*sU                                  : 3*sU, 
             3*sU + 2*sV                           : 3*sU + 3*sV, 
             3*sU + 3*sV + sP + sV                 : 3*sU + 3*sV + sP + 2*sV,
             3*sU + 3*sV + sP + 2*sV + 4*sL        : 3*sU + 3*sV + sP + 2*sV + 6*sL]


perm = np.r_[idx1, idx2, idx3]
P = sp.eye(len(perm)).tocsr()[perm,:]

MM = P.dot(MM.dot(P.T))
CC = P.dot(CC.dot(P.T))
KK = P.dot(KK.dot(P.T))

In [8]:
IA1 = sp.eye(sU + sV + sP + 2*sL)
IA2 = sp.eye(sU + 2*sV + 2*sL)

ZA1 = sp.coo_matrix((sU + sV + sP + 2*sL, sU + 2*sV + 2*sL))
ZA2 = sp.coo_matrix((sU + 2*sV + 2*sL, sU + 2*sV + 2*sL))

S1 = sp.bmat([[IA1, ZA1, ZA1]])
S2 = sp.bmat([[ZA1.T, IA2, ZA2]])
S3 = sp.bmat([[ZA1.T, ZA2, IA2]])

M11 = S1.dot(MM.dot(S1.T))
M12 = S1.dot(MM.dot(S2.T))
M13 = S1.dot(MM.dot(S3.T))

M21 = S2.dot(MM.dot(S1.T))
M22 = S2.dot(MM.dot(S2.T))
M23 = S2.dot(MM.dot(S3.T))

M31 = S3.dot(MM.dot(S1.T))
M32 = S3.dot(MM.dot(S2.T))
M33 = S3.dot(MM.dot(S3.T))

C11 = S1.dot(CC.dot(S1.T))
C12 = S1.dot(CC.dot(S2.T))
C13 = S1.dot(CC.dot(S3.T))

C21 = S2.dot(CC.dot(S1.T))
C22 = S2.dot(CC.dot(S2.T))
C23 = S2.dot(CC.dot(S3.T))

C31 = S3.dot(CC.dot(S1.T))
C32 = S3.dot(CC.dot(S2.T))
C33 = S3.dot(CC.dot(S3.T))

K11 = S1.dot(KK.dot(S1.T))
K12 = S1.dot(KK.dot(S2.T))
K13 = S1.dot(KK.dot(S3.T))

K21 = S2.dot(KK.dot(S1.T))
K22 = S2.dot(KK.dot(S2.T))
K23 = S2.dot(KK.dot(S3.T))

K31 = S3.dot(KK.dot(S1.T))
K32 = S3.dot(KK.dot(S2.T))
K33 = S3.dot(KK.dot(S3.T))

IIn = sp.bmat([[sp.eye(sU + sV), sp.coo_matrix((sU + sV, sV + 2*sL))]])

In [9]:
print(spla.norm(M22 - M33))
print(spla.norm(M23 - M32.T))

5.071135855983985e-13
0.0


In [10]:
print(spla.norm(C22 - C33))
print(spla.norm(C23 - C32.T))

5.659855294416393e-13
5.281667799315592


In [11]:
print(spla.norm(K22 - K33))
print(spla.norm(K23 + K32.T))

1.0249954520475414e-13
1.0693200423424283


In [12]:
Mk = np.zeros(M22.shape[0])
Mk[sU+sV: sU+2*sV]=1.0
tmp = (M22+M23+M32).dot(Mk)
print(la.norm(tmp))

0.0


In [13]:
!mkdir -p 1-ExtractMatrices

In [14]:
dest = '1-ExtractMatrices/'

PetscBinaryIO().writeMatSciPy(open(dest+'M1.dat','w'), M22)
PetscBinaryIO().writeMatSciPy(open(dest+'M2.dat','w'), M23)
PetscBinaryIO().writeMatSciPy(open(dest+'M3.dat','w'), M32)
PetscBinaryIO().writeMatSciPy(open(dest+'C1.dat','w'), C22)
PetscBinaryIO().writeMatSciPy(open(dest+'C2.dat','w'), C23)
PetscBinaryIO().writeMatSciPy(open(dest+'C3.dat','w'), C32)
PetscBinaryIO().writeMatSciPy(open(dest+'K1.dat','w'), K22)
PetscBinaryIO().writeMatSciPy(open(dest+'K2.dat','w'), K23)
PetscBinaryIO().writeMatSciPy(open(dest+'K3.dat','w'), K32)
PetscBinaryIO().writeMatSciPy(open(dest+'I.dat','w'), IIn)
PetscBinaryIO().writeVec(open(dest+'Mk.dat','w'), Mk)

In [16]:
PetscBinaryIO().writeVec(open(dest+'Rh12.dat','w'), np.ones(sU + sV))
PetscBinaryIO().writeVec(open(dest+'Mh12.dat','w'), np.ones(sU + sV))

In [17]:
solver.dt

0.02